In [ ]:
%matplotlib inline
import random, pylab, math, numpy

In this homework, we study path-sampling methods for the harmonic oscillator. The point is to improve on the naive path-sampling methods. We start with a preparation program that reminds us of the convenient way lists can manipulated in Python. Then, in the harmonic oscillator, we will use the harmonic Levy construction in two ways in the trap, then the free Levy construction. Finally, we apply the free and the harmonic Levy construction in an anharmonic potential down to quite low temperatures.

# A

In this section we provide a simple preparation program, shown here, and then analyze it.

In [ ]:
#
# Part one
#
L = range(10)
for k in range(10):
    print L
    L = L[3:] + L[:3]
print

#
# Part two
#

K = range(10)
for i in range(10):
    print K
    dummy = K.pop()
    K = [dummy] + K
print

#
# Part three
#

J = range(10)
for i in range(10):
    print K
    dummy = K.pop(0)
    K = K + [dummy]
print

#
# Part four
#

I = range(10)
weight = sum(a ** 2 for a in I)

## A1 

Run the Preparation program 1 and analyze its output.\:

## Questions

1. Explain what the "Part one" of the Preparation program 1 is good for. Attention, there are two aspects to this subquestion, they are related to list slicing and to list merging (concatenation)
2. Explain what the "Part two"of the Preparation program 1 does.
3. Explain what the "Part three"of the Preparation program 1 does.
4. Explain the concept of list comprehension underlying "Part four" of the Preparation program 1

# B

In this part, you explore sampling methods starting from naive_harmonic_path.py.

## B1 

Run the below program, a variant of the naive path sampling in the presence of a harmonic potential. Modify the sampling rate (at " if step%100 == 0: "), and the stepsize (at "delta = ...") but **not** the initial condition, and **not** the number of iterations.

1. Display a figure showing the best approximation you obtained for $\pi(x)$.
2. Discuss in a few words what you observe. 
3. Explain the three lines of this code containing the word "ProgType".

In [ ]:
def rho_free(x, y, beta):
    return math.exp(-(x - y) ** 2 / (2.0 * beta))

acc = 0
beta = 20.0
N = 80
dtau = beta / N
delta = 10.0
n_steps = 100000
x = [2.0] * N
data = []

for step in range(n_steps):
    k = random.randint(0, N - 1)
    knext, kprev = (k + 1) % N, (k - 1) % N
    x_new = x[k] + random.uniform(-delta, delta)
    old_weight = (rho_free(x[knext], x[k], dtau) * rho_free(x[k], x[kprev], dtau) *
                  math.exp(-0.5 * dtau * x[k] ** 2))
    new_weight = (rho_free(x[knext], x_new, dtau) * rho_free(x_new, x[kprev], dtau) *
                  math.exp(-0.5 * dtau * x_new ** 2))
    if random.uniform(0.0, 1.0) < new_weight / old_weight:
        x[k] = x_new
        acc += 1
    if step % 100 == 0:
        k = random.randint(0, N - 1)
        data.append(x[k])
        

In [ ]:
pylab.hist(data, bins=50, normed=True, label='QMC')
x_values = [0.1 * a for a in range (-30, 30)]
y_values = [math.sqrt(math.tanh(beta / 2.0)) / math.sqrt(math.pi) * 
            math.exp( - xx **2 * math.tanh( beta / 2.0)) for xx in x_values]
pylab.plot(x_values, y_values, label='exact')
pylab.xlabel('$x$')
pylab.ylabel('$\\pi(x)$ (normalized)')
pylab.axis([-3.0, 3.0, 0.0, 0.6])
pylab.legend()
ProgType = 'naive_harm_path'
pylab.title(ProgType + ' beta = ' + str(beta) + ', dtau = ' + str(dtau) + ', Nsteps = '+ str
(n_steps))
pylab.savefig(ProgType + str(beta) + '.png')
pylab.show()

## B2

Now modify your program by introducing using the Levy construction. For simplicity, the function levy_harmonic_path that you should use is provided below.

In [ ]:
def levy_harmonic_path(xstart, xend, dtau, N):
    x = [xstart]
    for k in range(1, N):
        dtau_prime = (N - k) * dtau
        Ups1 = (1.0 / math.tanh(dtau) + 1.0 / math.tanh(dtau_prime))
        Ups2 = (x[k - 1] / math.sinh(dtau) + xend / math.sinh(dtau_prime))
        x.append(random.gauss(Ups2 / Ups1, 1.0 / math.sqrt(Ups1)))
    return x

This function (which returns a list of N elements) should replace the naive construction for the N elements of the path (both xstart and xend should be the bead "0" of the path). Make sure that your path always has N elements: The final point x[N] is **not** used, as it is equal to x[0].

Run your program first for very small N. Notice that the position x[0] never changes. To fix this problem, use "Part 1" of the Preparation program 1 to wrap the path around the tau-axis after each Levy construction (replace "3" in the Preparation program 1 for example by "N / 2"). 

## Questions

1. Give a short explanation of what your program does.
2. Display a figure which shows that this program recovers the exact result for $\pi(x)$ for $\beta = 20$ (Don't forget to update ProgType.)
3. Explain whether this is a Markov-chain algorithm or not. 

## B3 

Now modify the program of section B2: since we know the distribution of x[0], we may sample from this distribution directly. Therefore, determine the standard deviation of the exact distribution $\pi(x) \propto \exp( - x^2 * \tanh(\beta / 2.0))$ (pay attention to the difference between standard deviation and variance).

Your new program should thus contain a line

In [ ]:
x[0] = random.gauss(0.0, sigma)

(or equivalent), where sigma is the standard deviation of $\pi(x)$, and this line should be followed by the harmonic Levy construction. No "wrapping" is needed now. Don't forget to update ProgType.

## Questions

1. Give a short explanation of what this program does
2. check that this program recovers the exact result for $\pi(x)$: display a figure showing the output for $\beta = 10$. 3. Explain whether this is a Markov-chain algorithm or not.

# C

In this section we go back to the wrapping algorithm of section B2 (**NOT B3**). We now use the free Levyconstruction in the harmonic potential. 

## C1

Add the function levy_free_path to the wrapping algorithm of section B2. For your convenience, it is given here (Note that it outputs a path of length N):

In [ ]:
def levy_free_path(xstart, xend, dtau, N):
    x = [xstart]
    for k in range(1, N):
        dtau_prime = (N - k) * dtau
        x_mean = (dtau_prime * x[k - 1] + dtau * xend) / (dtau + dtau_prime)
        sigma = math.sqrt(1.0 / (1.0 / dtau + 1.0 / dtau_prime))
        x.append(random.gauss(x_mean, sigma))
    return x

Check that for a path obtained from the Free levy construction, the statistical weight is given by the well-known exponential of the "Feynman action" $$\exp(- \sum \delta_\tau \sum_i V(i))$$ 
which translates in python into the following (see Preparation program, Part 4):

In [ ]:
Weight_trott = math.exp(sum(-a **2/ 2.0 * dtau for a in x))

Therefore, replace the harmonic Levy construction of section B2 by the free Levy construction, but new paths should be accepted with the Metropolis algorithm.
The program of this section should

1. Propose a new path between x[0] and x[0], from Leey_free_path.
2. Compute its Weight (modify the above line... x[..] will have to be replaced).
3. Accept the new path with probability min(1, Weight_new/ Weight_old).
4.. Update the Weight (if move accepted).
5. Update the path (if move accepted). (**Attention**: here, the syntax "x = x_new" leads to a disaster. Using the python construct "x = x_new[:]" is much better...).
6. Wrap the path, as in section B2.

Check your program for small values of $\beta$. At large $\beta$ (around $\beta = 20$), the acceptance probability falls to zero, but this can be cured in a single line:

In [ ]:
x_new = levy_free_path(x[0], x[Ncut], dtau, Ncut) + x[Ncut:]

...where Ncut is chosen suitably. We thus construct a new path between 0 and Ncut-1, and the old path from Ncut through N - 1. 

## Question

1. Display the result of your program for $\beta = 20$ (do not forget to update ProgType)

## C2

Adapt the program of section C1 for the anharmonic potential implemented by the function 
$$V(x) = x^2 / 2 + cubic * x^3 + quartic * x^4$$
with

cubic = -1, quartic = 1.

Modify your program so that it samples the anharmonic path integral at $\beta = 20$, starting from 

1. the free Levy construction.
1. the harmonic Levy construction.

Naturally, the Feynman action (the Weight) is different in both cases, because a term counted in the "measure" of the probability distribution should not be counted in the "statistical weight" of the paths. If possible, introduce a variable "LevyType" to choose the potential and to modify ProgType

## Question

1. Display the obtained distributions $\pi(x)$. (They should be very similar, at sufficiently small dtau.)